# Extrair informações dos relatórios emitidos pelo sistema de envio de mensagens
## ENTRADA: Relatório em html
## SAÍDA: Qnt envio sucesso, qnt envio falha, registro dos números inválidos em .txt

In [2]:
import pandas as pd
import os

def lerNumerosInvalidos():
    with open("invalidos.txt", "r") as aq:
        listaInvalidos = aq.readlines()
    return listaInvalidos



def lerArquivosVerificados():
    with open("arquivosVerificados.txt", "r") as aq:
        listaArquivos = aq.readlines()
    return listaArquivos



def salvarNumerosInvalidos(lista):
    with open("invalidos.txt", "a") as x:
        for i in lista:
            x.write(i +'\n')

            

def salvarNumerosValidos(lista):
    arquivo = r"G:\.shortcut-targets-by-id\1btUlJgqPrq34yV-xFURKWluZW6W4AQB2\RAPIDO\numerosValidos.txt"
    with open(arquivo, "w") as x:
        for i in lista:
            x.write(i)



def salvarArquivoVerificado(nomeArq):
    with open("arquivosVerificados.txt", "a") as arq:
        arq.write(nomeArq.split("/")[1]+'\n')
            
            

#VERIFICA SE 'numero' ESTAR CONTIDO NA LISTA DE NÚMEROS INVÁLIDOS 'listaNumeros'
##ENTRADA: 
### (listaNumeros - list) - Lista de números inválidos salvos em .txt
### (numero - string) - numero para buscar ocorrência na lista
##SAÍDA:
### True ou False
def numeroSalvo(listaNumeros, numero):
    for item in listaNumeros:
        if numero in item:
            return True    
        
    return False



#VERIFICA SE O RELATÓRIO JÁ FOI ANALISADO BUSCANDO O NOME DO RELATÓRIO NA LISTA DE ARQUIVOS VERIFICADOS
##ENTRADA:
### (arquivoHtml - string) - nome do relatório em html
##SAÍDA:
### True ou False
def arquivoVerificado(arquivoHtml):
    listaVerificados = lerArquivosVerificados()
    for arq in listaVerificados:
        #comparar apenas o nome do arquivo sem a extensão
        if arq.split(".")[0] in arquivoHtml:
            return True
    
    return False



#RETORNA UMA LISTA COM OS NOMES DOS ARQUIVOS DE MESMA EXTENSÃO EM UMA PASTA
##ENTRADA:
### (caminhoArquivos - string) - Local dos arquivos 
### (extensaoArquivo - string) - extensão do arquivo  EX: txt, xlsx 
##SAÍDA:
###Lista com os nomes dos arquivos
def listarNomeArquivos(caminhoArquivos, extensaoArquivo):
    listaTodosArquivos = os.listdir(caminhoArquivos) #lista o nome de todos arquivos da pasta
    nomesArquivos = []
    for arq in listaTodosArquivos:
        if extensaoArquivo in arq: #lista arquivos de determinada extensão
            nomesArquivos.append(caminhoArquivos + "/" + arq)
    
    return ordenarLista(nomesArquivos)



def ordenarLista(listaNomeArquivo):
    lista_datas = []
    listaOrdenada = []
    
    for arquivo in listaNomeArquivo:
        data = os.path.getmtime(arquivo)
        lista_datas.append((data, arquivo))
        
    lista_datas.sort(reverse=True)
    
    for item in lista_datas:
        listaOrdenada.append(item[1])
    
    return listaOrdenada



#EXTRAI INFORMAÇÕES DO RELATÓRIO 'HTML' E GERA LISTA DE NÚMEROS INVÁLIDOS
##ENTRADA:
### (arquivoHtml - string) - caminho completo do relatório em html emitido pelo sistema
##SAÍDA:
### (listaInvalidos - list) - lista de números inválidos 
### (qntInclusoes - int) - Quantidade de novos números inválidos incluidos na lista 
def informacoesRelatorioHtml(arquivoHtml):
    
    df = pd.read_html(arquivoHtml)
    listaSalva = lerNumerosInvalidos()
    data = df[0]["Data"][0].split(" ")[0]
    hora = df[0]["Data"][0].split(" ")[1]
    qntTotal = df[0]["Total de Número"][0]
    qntSucesso = df[0]["Bem sucedido"][0]
    qntFalha = df[0]["Falhou"][0]
    
    listaInvalidos = []
    qntInclusoesInvalidas = 0
    qntRemovidos = 0
    
    for i, item in enumerate(df[2]["Número"]):
        n = str(df[2]["Número"][i])[-8:]
        if numeroSalvo(listaSalva, n):
            if df[2]["Status"][i] == "Enviado":
                listaSalva.pop(i)
                qntRemovidos += 1
        elif "Falhou" == df[2]["Status"][i]:
            qntInclusoesInvalidas += 1
            listaInvalidos.append(n)
     
    return listaInvalidos, qntInclusoesInvalidas, qntRemovidos, qntTotal, qntSucesso, qntFalha, data, hora



def gerarListaEnvio():
    
    arq =  r"G:\.shortcut-targets-by-id\1btUlJgqPrq34yV-xFURKWluZW6W4AQB2\RAPIDO\verificar.txt"
    a = "novosNumeros.txt"
    with open(arq, "r") as aq:
        listaNumeros = aq.readlines()
        
    listaInvalidos = lerNumerosInvalidos()
    listaEnvio = []
    qnt = 0
    
    for numero in listaNumeros:
        if not(numeroSalvo(listaInvalidos, numero[-8:])):
            qnt += 1
            listaEnvio.append(numero)
    
    salvarNumerosValidos(listaEnvio)
    print(f"Total números: {len(listaNumeros)}  -  Válidos {qnt}")



def executar():
    
    listaInvalidos = []
    data = ""
    hora = ""
    qntTotal = 0
    qntSucesso = 0
    qntFalha = 0
    qntInclusoes = 0
    qntRemocoes = 0
    qntTotalInclusoes = 0
    qntTotalRemocoes = 0
    
    localRelatorios = r"G:\.shortcut-targets-by-id\1-DgZ1HKaj2QzzO9O_J5BQlhJUh1WsGst\RelatorioWA"
    extensao = "html"
    arquivosHtml = listarNomeArquivos(localRelatorios, extensao)
    print(f"Quantidade de relatórios: {len(arquivosHtml)}")
    
    for arquivo in arquivosHtml:
        if not(arquivoVerificado(arquivo)):
            listaInvalidos, qntInclusoes, qntRemocoes, qntTotal, qntSucesso, qntFalha, data, hora = informacoesRelatorioHtml(arquivo)
            qntTotalInclusoes += qntInclusoes
            qntTotalRemocoes += qntRemocoes
            salvarNumerosInvalidos(listaInvalidos)
            salvarArquivoVerificado(arquivo)
            print(f"{arquivo} - {data} {hora} ")
            print(f"Total: {qntTotal} -  Sucesso: {qntSucesso} - Falhas: {qntFalha}")
            print(f"Incluído: {qntInclusoes} - Removido: {qntRemocoes}")
        
    print(f"Total inválidos inclusos: {qntTotalInclusoes} - Total inválidos removidos: {qntTotalRemocoes}")




In [6]:
#rel = r"G:/.shortcut-targets-by-id/1-DgZ1HKaj2QzzO9O_J5BQlhJUh1WsGst/RelatorioWA/27032023125937867.html"
#listaInvalidos, qntInclusoesInvalidas, qntRemovidos, qntTotal, qntSucesso, qntFalha, data, hora = informacoesRelatorioHtml(rel)
#print(qntInclusoesInvalidas, qntRemovidos, qntTotal, qntSucesso, qntFalha)
#gerarListaEnvio()
executar()

Quantidade de relatórios: 514
Total inválidos inclusos: 0 - Total inválidos removidos: 0
